<a href="https://colab.research.google.com/github/Juaano28/ProyectoFinal_TAM_CEM/blob/main/Proyecto_TAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Promedios diarios 2019, 2020, 2021, 2022 y 2023

In [ ]:
import pandas as pd
import numpy as np

# Usa raw string para evitar problemas con las barras invertidas y agrega la extensión .csv
df = pd.read_csv(r'/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Datasets/data_Bogota_BOGOTA.csv')

# Extraer la fecha de 'fecha_medicion'
df['fecha'] = pd.to_datetime(df['fecha_medicion'].str.split(" ").str[0], format='%Y/%m/%d')

# --- New Filtering Step: Identify probes with data in 2023 ---
# Filter data for the year 2023
df_2023 = df[df['fecha'].dt.year == 2023]

# Get the unique probe locations that have data in 2023
probes_with_data_in_2023 = df_2023['ubicacion_sonda'].unique()

print(f"Número de sondas con datos en 2023: {len(probes_with_data_in_2023)}")
print("Sondas con datos en 2023:", probes_with_data_in_2023.tolist())

# Filter the original DataFrame to include only these probes
df_filtered_probes = df[df['ubicacion_sonda'].isin(probes_with_data_in_2023)].copy()
# -----------------------------------------------------------


# Filter the data for the years 2019, 2020, 2021, 2022, 2023 and 2024
# NOTE: This year filtering is now applied to df_filtered_probes
df_filtered = df_filtered_probes[df_filtered_probes['fecha'].dt.year.isin([2019, 2020, 2021, 2022, 2023, 2024])]
print("Número de registros (para sondas con data en 2023) para 2019-2024:", len(df_filtered))


# Crear un DataFrame con la serie completa de fechas para los años 2019, 2020, 2021, 2022, 2023 y 2024
fechas_completas = pd.date_range(start='2019-01-01', end='2024-12-31', freq='D')
df_fechas = pd.DataFrame({'serie': fechas_completas})

# Agrupar los datos por 'fecha' y 'ubicacion_sonda' y calcular el promedio diario
# Use df_filtered for aggregation
df_promedio = df_filtered.groupby(['fecha', 'ubicacion_sonda'])['intensidad_campo'].mean().reset_index()

# Pivotear la tabla para que cada 'ubicacion_sonda' sea una columna
df_pivot = df_promedio.pivot(index='fecha', columns='ubicacion_sonda', values='intensidad_campo').reset_index()

# Renombrar la columna 'fecha' a 'serie'
df_pivot = df_pivot.rename(columns={'fecha': 'serie'})

# Hacer un merge con el DataFrame de fechas para incluír todos los días
df_final = pd.merge(df_fechas, df_pivot, on='serie', how='left')

# Llenar los datos faltantes con "N/A"
# This will fill days within 2019-2024 where a probe had no data on that specific day,
# or probes that had data in 2023 but not in other years within the range.
df_final = df_final.fillna("N/A")

# Convertir la columna 'serie' a formato de texto (YYYY-MM-DD)
df_final['serie'] = df_final['serie'].dt.strftime('%Y-%m-%d')

# Guardar en Google Drive con un nuevo nombre de archivo (optional, can overwrite or save with a new name)
output_path = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Bog_promedios_diarios_2019_2024_filtered_2023_probes.csv'
df_final.to_csv(output_path, index=False)
print(f"\nPromedios diarios guardados en: {output_path}")

print(df_final.head())
print("\nShape of df_final:", df_final.shape)

# Coordenadas por sonda

In [ ]:
import pandas as pd

# Usa raw string para evitar problemas con las barras invertidas y agrega la extensión .csv
df = pd.read_csv(r'/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Datasets/data_Bogota_BOGOTA.csv')

# Selección de las columnas relevantes
df_seleccion = df[['ubicacion_sonda', 'georreferencia']]

# Eliminar duplicados
df_unico = df_seleccion.drop_duplicates(subset=['ubicacion_sonda']).copy()

# Renombrar las columnas:
df_unico = df_unico.rename(columns={
    'ubicacion_sonda': 'posiciones',
    'georreferencia': 'coordenadas'
})

# Guardar en Google Drive
df_unico.to_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Zip_ubicaciones_con_coordenadas.csv', index=False)
print(df_unico.head())

# Visualización

In [ ]:
import pandas as pd

# Cargar el archivo CSV con los promedios diarios
df_final = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Bogotá/Bog_promedios_diarios_2019_2024.csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure df_final is available (from cell n3nrda8PCljf or the cell that creates it)
# df_final should contain the data after initial daily averaging and filling with "N/A"
# Ensure df_final_imputed is available (from cell 52b51111 or the cell that performs imputation)
# df_final_imputed should contain the data after handling "N/A" and imputation

if 'df_final' not in locals():
    print("df_final not found. Please run the cell that creates df_final (e.g., n3nrda8PCljf).")
elif 'df_final_imputed' not in locals():
    print("df_final_imputed not found. Please run the cell that performs imputation (e.g., 52b51111).")
else:
    # Make a temporary copy of df_final and convert "N/A" to NaN to find last non-null dates
    df_temp_dates = df_final.replace("N/A", np.nan).copy()

    # Ensure 'serie' is datetime
    df_temp_dates['serie'] = pd.to_datetime(df_temp_dates['serie'])
    df_temp_dates = df_temp_dates.set_index('serie')

    # Find the last non-null index (date) for each column
    last_valid_dates = df_temp_dates.apply(lambda x: x.last_valid_index())

    # Filter probes whose last valid date is within the year 2023
    probes_data_until_2023_series = last_valid_dates[
        (last_valid_dates.dt.year == 2023)
    ]

    probes_data_until_2023_names = probes_data_until_2023_series.index.tolist()


    print("Sondas con datos medidos (no imputados) hasta algún punto dentro del año 2023:")
    if not probes_data_until_2023_series.empty:
        # Print the probes and their last valid date in 2023
        print(probes_data_until_2023_series)

        # --- Visualization Step ---
        print("\nGenerating time series plots for these probes...")

        # Filter the imputed DataFrame to include only these probes and 'serie'
        if 'serie' in df_final_imputed.columns:
             cols_to_plot = ['serie'] + probes_data_until_2023_names
             df_to_plot = df_final_imputed[cols_to_plot].copy()
        else:
             # Handle case where 'serie' might not be in df_final_imputed (unlikely if following steps)
             print("Column 'serie' not found in df_final_imputed. Cannot plot time series.")
             df_to_plot = None


        if df_to_plot is not None:
            # Ensure 'serie' is datetime for plotting
            df_to_plot['serie'] = pd.to_datetime(df_to_plot['serie'])

            # Set 'serie' as index for easier plotting
            df_to_plot = df_to_plot.set_index('serie')

            # Determine the number of probes to plot
            num_probes_to_plot = len(probes_data_until_2023_names)

            # Create subplots for each probe
            n_cols = min(num_probes_to_plot, 4) # Use max 4 columns
            n_rows = (num_probes_to_plot + n_cols - 1) // n_cols

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
            # Handle case where there's only one subplot
            if num_probes_to_plot == 1:
                axes = [axes]
            else:
                axes = axes.flatten()

            # Plot time series for each probe
            for i, probe_name in enumerate(probes_data_until_2023_names):
                ax = axes[i]
                ax.plot(df_to_plot.index, df_to_plot[probe_name])
                ax.set_title(f'Serie de Tiempo: {probe_name}')
                ax.set_xlabel('Fecha')
                ax.set_ylabel('Intensidad de Campo')
                ax.grid(True)
                plt.xticks(rotation=45, ha='right') # Rotate x-axis labels

            # Hide any unused subplots
            for j in range(num_probes_to_plot, len(axes)):
                fig.delaxes(axes[j])

            plt.tight_layout()
            plt.suptitle('Series de Tiempo para Sondas con Datos hasta 2023', y=1.02)
            plt.show()

        # --- End Visualization Step ---

    else:
        print("No se encontraron sondas cuya última fecha con datos medidos sea en 2023.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Import seaborn for boxplots

# Ensure df is available (from cell n3nrda8PCljf or the cell that loads original data)
if 'df' not in locals():
    print("df not found. Please run the cell that loads the original DataFrame (e.g., n3nrda8PCljf).")
else:
    # --- FIX: Create the 'fecha' column if it doesn't exist ---
    if 'fecha' not in df.columns:
         df['fecha'] = pd.to_datetime(df['fecha_medicion'].str.split(" ").str[0], format='%Y/%m/%d')
    # -----------------------------------------------------------

    # --- Identify probes with data up to 2023 (reusing logic from 2172802b) ---
    # Create a temporary DataFrame to find last non-null dates, focusing on relevant columns
    df_temp_dates = df[['fecha', 'ubicacion_sonda', 'intensidad_campo']].copy()

    # Pivot the data to easily find the last non-null date per probe
    df_pivot_temp = df_temp_dates.pivot_table(index='fecha', columns='ubicacion_sonda', values='intensidad_campo', aggfunc='mean') # Use mean in case of multiple measurements per day

    # Find the last non-null index (date) for each column
    last_valid_dates = df_pivot_temp.apply(lambda x: x.last_valid_index())

    # Filter probes whose last valid date is within the year 2023
    probes_data_until_2023_series = last_valid_dates[
        (last_valid_dates.dt.year == 2023)
    ]

    probes_data_until_2023_names = probes_data_until_2023_series.index.tolist()

    print("Sondas con datos medidos (no imputados) hasta algún punto dentro del año 2023:")
    if not probes_data_until_2023_series.empty:
        print(probes_data_until_2023_series)
    else:
        print("No se encontraron sondas cuya última fecha con datos medidos sea en 2023.")
        probes_data_until_2023_names = [] # Ensure the list is empty if no probes found
    # --------------------------------------------------------------------------


    # --- Filter the original DataFrame for these probes and generate boxplots ---
    if probes_data_until_2023_names:
        print("\nGenerando boxplots para las sondas con datos hasta 2023:")
        # Filter the original df to include only records for these probes
        df_filtered_for_boxplot = df[df['ubicacion_sonda'].isin(probes_data_until_2023_names)].copy()

        # Determine the number of probes to plot
        num_probes_to_plot = len(probes_data_until_2023_names)

        # Create subplots for each probe's boxplot
        n_cols = min(num_probes_to_plot, 4) # Use max 4 columns
        n_rows = (num_probes_to_plot + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
        # Handle case where there's only one subplot
        if num_probes_to_plot == 1:
            axes = [axes]
        else:
            axes = axes.flatten()

        # Generate boxplot for each probe
        for i, probe_name in enumerate(probes_data_until_2023_names):
            ax = axes[i]
            # Filter data for the current probe
            data_to_plot = df_filtered_for_boxplot[df_filtered_for_boxplot['ubicacion_sonda'] == probe_name]
            sns.boxplot(y='intensidad_campo', data=data_to_plot, ax=ax)
            ax.set_title(f'Boxplot: {probe_name}')
            ax.set_ylabel('Intensidad de Campo')
            ax.set_xlabel('') # No x-label needed for single boxplot per subplot
            ax.grid(True)

        # Hide any unused subplots
        for j in range(num_probes_to_plot, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.suptitle('Boxplots de Intensidad de Campo para Sondas con Datos hasta 2023', y=1.02)
        plt.show()

    else:
        print("\nNo hay sondas para generar boxplots (ninguna con datos hasta 2023).")

# Preprocesamiento

## Manejo de valores faltantes

### Subtask:
Decidir cómo manejar los valores "N/A" (que ya hemos reemplazado por NaN) en el DataFrame `df_final`.


In [ ]:
import numpy as np

# Calculate the number of missing values per column
missing_counts = df_final.isnull().sum()

# Calculate the percentage of missing values per column
missing_percentages = (missing_counts / len(df_final)) * 100

print("Cantidad de valores NaN por columna:")
print(missing_counts)
print("\nPorcentaje de valores NaN por columna:")
print(missing_percentages)

# Define the threshold for missing values percentage
threshold = 60

# Identify columns to drop
columns_to_drop = missing_percentages[missing_percentages > threshold].index.tolist()

if columns_to_drop:
    print(f"\nColumnas a eliminar (más del {threshold}% de valores faltantes):")
    print(columns_to_drop)

    # Drop the columns from the DataFrame
    df_final = df_final.drop(columns=columns_to_drop)
    print("\nColumnas restantes en el DataFrame:")
    print(df_final.columns.tolist())
else:
    print(f"\nNo hay columnas con más del {threshold}% de valores faltantes.")

# Print the number of missing values again after dropping columns
print("\nCantidad de valores NaN por columna después de eliminar columnas:")
print(df_final.isnull().sum())

# --- Diagnostic Print: Dtypes after dropping columns ---
print("\nDtypes of df_final after dropping columns:")
print(df_final.dtypes)
# -----------------------------------------------------

In [ ]:
import numpy as np
import pandas as pd

# Implementar imputación con la mediana para los valores faltantes
# Operando sobre el DataFrame df_final (que es la salida de la celda anterior y podría tener columnas eliminadas)

# Make a copy
df_final_imputed = df_final.copy()

# Convert "N/A" to NaN
df_final_imputed = df_final_imputed.replace("N/A", np.nan)

# Separate the 'serie' column from the numeric columns
if 'serie' in df_final_imputed.columns:
    df_serie = df_final_imputed['serie'].copy() # Use copy to avoid SettingWithCopyWarning
    df_numeric_only = df_final_imputed.drop(columns=['serie']).copy() # Use copy
else:
    # Handle case where 'serie' might have been dropped (unlikely but safe)
    df_serie = None
    df_numeric_only = df_final_imputed.copy() # Use copy


# --- Ensure numeric columns are purely numeric ---
# Convert all columns in df_numeric_only to numeric, coercing errors
for col in df_numeric_only.columns:
    df_numeric_only[col] = pd.to_numeric(df_numeric_only[col], errors='coerce')
# -------------------------------------------------


# --- Simplified Median Imputation on Numeric Columns ---

# Calculate the median for the numeric columns (NaNs resulting from coerce will be ignored)
# Use .select_dtypes(include=np.number) just in case, though all should be numeric now
median_values = df_numeric_only.select_dtypes(include=np.number).median()

# Fill NaN values in the numeric DataFrame using the calculated medians
# Pandas aligns by column names automatically.
df_numeric_only = df_numeric_only.fillna(median_values)

# --- End Simplified Median Imputation ---


# Combine the imputed numeric columns and the 'serie' column back
if df_serie is not None:
    # Reset index of both dataframes before concatenating to avoid index alignment issues
    df_serie_reset = df_serie.reset_index(drop=True)
    df_numeric_only_reset = df_numeric_only.reset_index(drop=True)

    # Create the final DataFrame by joining the 'serie' column back
    # Ensure alignment by index (which should be default after reset_index)
    df_final_imputed = pd.concat([df_serie_reset, df_numeric_only_reset], axis=1)
else:
    # If 'serie' was not present, the imputed numeric_only is the final DataFrame
    df_final_imputed = df_numeric_only


# Verificar si quedan valores NaN después de la imputación con la mediana
print("\nCantidad de valores NaN por columna después de la imputación con la mediana:")
print(df_final_imputed.isnull().sum())

# Display the first few rows of the imputed DataFrame
print("\nPrimeras filas del DataFrame imputado con la mediana:")
display(df_final_imputed.head())

# NOTE: The subsequent cell (3c8b12c3) currently performs ffill and bfill.
# If you intend to use only median imputation, you might skip or modify that cell.

In [ ]:
# Aplicar forward fill y luego backward fill para los NaN restantes
df_final_imputed = df_final_imputed.fillna(method='ffill').fillna(method='bfill')

# Asegurarse de que todas las columnas numéricas sean de tipo float
for column in df_final_imputed.columns:
    if column != 'serie':
        df_final_imputed[column] = df_final_imputed[column].astype(float)


# Verificar si quedan valores NaN después del ffill y bfill
print("\nCantidad de valores NaN por columna después de ffill y bfill:")
print(df_final_imputed.isnull().sum())

## Normalización o escalado de datos

### Subtask:
Escalar los valores de intensidad de campo en el DataFrame `df_final_interpolated` a un rango específico.


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical columns for scaling, excluding the 'serie' column
numerical_cols = df_final_imputed.columns.drop('serie')
df_to_scale = df_final_imputed[numerical_cols]

# Instantiate MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler to the data and transform the data
scaled_data = scaler.fit_transform(df_to_scale)

# Create a new DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=numerical_cols)

# Add the 'serie' column back to the scaled DataFrame
df_scaled['serie'] = df_final_imputed['serie']

# Ensure all numerical columns in df_scaled are float
for column in df_scaled.columns:
    if column != 'serie':
        df_scaled[column] = df_scaled[column].astype(float)

# Display the first few rows of the scaled DataFrame
display(df_scaled.head())

## Creación de secuencias para lstm

### Subtask:
Crear pares de entrada-salida en formato de secuencia para el modelo LSTM utilizando el DataFrame escalado `df_scaled`.


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

def create_sequences(data, n_steps):
    X, y = [], []
    # data should only contain the numerical columns at this point

    for i in range(len(data) - n_steps):
        # Extract the input sequence (features)
        seq_x = data.iloc[i:(i + n_steps), :].values.astype(np.float32)
        # Extract the output value(s) (targets)
        seq_y = data.iloc[i + n_steps, :].values.astype(np.float32)

        X.append(seq_x)
        y.append(seq_y)

    # Convert the list of arrays to a single numpy array with specified dtype
    # Ensure resulting arrays are float32
    X_array = np.array(X, dtype=np.float32)
    y_array = np.array(y, dtype=np.float32)

    # --- Diagnostic Prints ---
    # print("Dtypes of df_scaled_numerical before creating sequences:")
    # print(df_scaled_numerical.dtypes)
    # -------------------------

    # --- Diagnostic Prints ---
    print("\nDtype of resulting X array after create_sequences:", X_array.dtype)
    print("Dtype of resulting y array after create_sequences:", y_array.dtype)
    # -------------------------


    return X_array, y_array

# Define the number of time steps
n_steps = 60

# Select only the numerical columns from the scaled DataFrame
df_scaled_numerical = df_scaled.drop(columns=['serie']).copy()

# Create sequences and targets using only the scaled numerical data
X, y = create_sequences(df_scaled_numerical, n_steps)


print("\nShape of input sequences (X):", X.shape)
print("Shape of output targets (y):", y.shape)

# --- Explicitly ensure X and y are float32 before splitting ---
X = X.astype(np.float32)
y = y.astype(np.float32)
# -------------------------------------------------------------


# Split the data into training and a temporary set (for validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the shapes and dtypes of the resulting sets
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

print("\nDtypes of training, validation, and test sets after splitting:")
print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)
print("X_val dtype:", X_val.dtype)
print("y_val dtype:", y_val.dtype)
print("X_test dtype:", X_test.dtype)
print("y_test dtype:", y_test.dtype)

## División de datos

### Subtask:
Dividir el conjunto de datos en conjuntos de entrenamiento, validación y prueba.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and a temporary set (for validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

## Reestructuración para lstm

### Subtask:
Asegurar que los datos tengan la forma correcta para la entrada del modelo LSTM (generalmente 3D: [muestras, pasos de tiempo, características]).


In [ ]:
import pandas as pd

# Ensure df_scaled is available (it was created in cell 418f686c)
# If you restarted the notebook, you might need to re-run the preprocessing cells.

# Define the path where you want to save the CSV file in Google Drive
output_path = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Bog_datos_preprocesados_final.csv'

# Save the df_scaled DataFrame to a CSV file
# Set index=False to avoid writing the DataFrame index as a column
df_scaled.to_csv(output_path, index=False)

print(f"Base de datos preprocesada final guardada en: {output_path}")

## Summary:

### Data Analysis Key Findings

*   Initial data contained a significant number of missing values (NaNs) across almost all sensor columns, ranging from approximately 426 to 1040 per column.
*   A two-step imputation process involving linear interpolation followed by forward and backward fill successfully handled all missing values.
*   The numerical intensity field values were scaled to a range of (0, 1) using `MinMaxScaler`.
*   Input sequences ($X$) and output targets ($y$) were created with a sequence length of 60 time steps. The resulting shapes are $X$: $(1036, 60, 14)$ and $y$: $(1036, 14)$, indicating 1036 samples, each with 60 time steps and 14 features, and corresponding targets for the 14 sensors.
*   The data was split into training ($X\_train$, $y\_train$), validation ($X\_val$, $y\_val$), and test ($X\_test$, $y\_test$) sets with shapes of $(725, 60, 14)$, $(155, 60, 14)$, and $(156, 60, 14)$ respectively for the input sequences. The corresponding output shapes are $(725, 14)$, $(155, 14)$, and $(156, 14)$.
*   The input data arrays are already in the required 3D format ([samples, time steps, features]) for LSTM models.

### Insights or Next Steps

*   The preprocessed data is now ready for training an LSTM model for forecasting intensity fields.
*   Consider exploring different sequence lengths or data scaling methods in future iterations to optimize model performance.


## Definición y entrenamiento del modelo LSTM

### Subtask:
Definir la arquitectura del modelo LSTM, compilarlo y entrenarlo utilizando los datos preprocesados.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np

# NOTE: Ensure the preceding preprocessing cells (loading data, handling missing values,
# scaling, and creating sequences/splitting data) have been executed to load the latest
# preprocessed data into X_train, y_train, X_val, y_val.

# Define the LSTM model
model = Sequential()
# Manual Refinement Step 1: Adjusting LSTM units from 50 to 75
# Manual Refinement Step 2: Add a second LSTM layer
model.add(LSTM(units=75, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3)) # Manual Refinement Step 4: Adjust dropout rate
model.add(LSTM(units=75, activation='relu')) # Second LSTM layer, no return_sequences=True
model.add(Dropout(0.3)) # Manual Refinement Step 4: Adjust dropout rate
model.add(Dense(units=y_train.shape[1])) # Output layer with the number of target features

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Summarize the model
model.summary()

# Manual Refinement Step 3: Modify training parameters (epochs and batch size)
print("\nTraining with adjusted parameters (epochs=100, batch_size=64)...")
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), verbose=1)

print("\nTraining complete.")

## Evaluación del modelo

### Subtask:
Evaluar el rendimiento del modelo entrenado utilizando el conjunto de prueba.

In [ ]:
# --- Diagnostic Prints ---
print("\nDtypes of test data before model.evaluate:")
print("X_test dtype:", X_test.dtype)
print("y_test dtype:", y_test.dtype)
# -------------------------

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test set loss: {loss}')

## Visualización del historial de entrenamiento

### Subtask:
Visualizar la pérdida de entrenamiento y validación a lo largo de las épocas.

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss during Training')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

## Visualización de Predicciones vs. Valores Reales

### Subtask:
Generar predicciones utilizando el modelo entrenado en el conjunto de prueba y compararlas visualmente con los valores reales.

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

print("Shape of predictions (y_pred):", y_pred.shape)
print("Shape of actual values (y_test):", y_test.shape)

In [ ]:
import matplotlib.pyplot as plt

# Get the column names for the probes from the numerical scaled DataFrame
probe_names = df_scaled_numerical.columns.tolist()

# Determine the number of probes
num_probes = len(probe_names)

# Create a figure and a set of subplots
# Adjust the layout (nrows, ncols) and figsize based on the number of probes
# We'll try a 4x4 grid as an example, adjust as needed
n_cols = 4
n_rows = (num_probes + n_cols - 1) // n_cols # Calculate number of rows needed

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# Iterate through each probe and create a subplot
for i in range(num_probes):
    ax = axes[i]
    probe_name = probe_names[i]

    # Plot actual values and predictions for the current probe
    ax.plot(y_test[:, i], label='Valores Reales')
    ax.plot(y_pred[:, i], label='Predicciones')

    ax.set_title(f'Sonda: {probe_name}')
    ax.set_xlabel('Paso de Tiempo (en conjunto de prueba)')
    ax.set_ylabel('Intensidad de Campo (Escalada)')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(num_probes, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout() # Adjust layout to prevent titles/labels overlapping
plt.suptitle('Predicciones vs. Valores Reales por Sonda (Conjunto de Prueba)', y=1.02) # Add a main title
plt.show()

## Métricas de Error Específicas

### Subtask:
Calcular métricas de error (RMSE, MAE, MAPE) para evaluar el rendimiento del modelo en el conjunto de prueba.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE) en el conjunto de prueba: {rmse}')

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE) en el conjunto de prueba: {mae}')

# Validación cruzada
Implementar validación cruzada para series de tiempo en el modelo LSTM utilizando los datos preprocesados, mostrando el código y explicando los resultados.

## Preparar los datos para validación cruzada

### Subtask:
Asegurarse de que los datos estén ordenados cronológicamente y listos para ser divididos en pliegues de entrenamiento y validación secuenciales.


In [ ]:
# Ensure the DataFrame is sorted by date
df_scaled = df_scaled.sort_values(by='serie').reset_index(drop=True)

# Separate features (X) and targets (y)
# Exclude the 'serie' column from features
features = df_scaled.drop(columns=['serie'])
targets = df_scaled.drop(columns=['serie']) # For multi-output LSTM, targets are the same features shifted

# Convert to NumPy arrays
X_sorted = features.values
y_sorted = targets.values

# Print shapes to verify
print("Shape of sorted features (X_sorted):", X_sorted.shape)
print("Shape of sorted targets (y_sorted):", y_sorted.shape)

In [ ]:
# Define the number of time steps (using the same n_steps as before)
n_steps = 60

# Create sequences and targets using the sorted data
# X will be the input sequences (n_steps) and y will be the next step (1)
X_cv, y_cv = [], []

for i in range(len(X_sorted) - n_steps):
    # Input sequence: n_steps data points for all features
    seq_x = X_sorted[i:(i + n_steps), :]
    # Output target: the next data point for all features
    seq_y = y_sorted[i + n_steps, :]

    X_cv.append(seq_x)
    y_cv.append(seq_y)

# Convert the list of arrays to a single numpy array
X_cv = np.array(X_cv)
y_cv = np.array(y_cv)

# Print the shapes to verify
print("Shape of input sequences for CV (X_cv):", X_cv.shape)
print("Shape of output targets for CV (y_cv):", y_cv.shape)

## Definir la estrategia de validación cruzada

### Subtask:
Configurar un iterador de validación cruzada para series de tiempo (como `TimeSeriesSplit` de scikit-learn) que defina cómo se dividirán los datos en pliegues.


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Define the number of splits for TimeSeriesSplit
# The number of splits determines how many train/test splits are created.
# With n_splits=5, you will have 5 different splits.
# The test set size for each split is approximately total_samples / (n_splits + 1)
# Given our total number of sequences is 1036, n_splits=5 seems reasonable,
# resulting in test sets of around 1036 / 6 = ~172 samples.
n_splits = 5

# Instantiate TimeSeriesSplit
# TimeSeriesSplit provides train/test indices to split time series data samples
# that are observed at fixed time intervals, in train/test sets.
# In each split, the training set is always before the test set.
# The test sets are successive samples that are supersets of the previous test sets.
tscv = TimeSeriesSplit(n_splits=n_splits)

print(f"TimeSeriesSplit configured with {n_splits} splits.")

## Implementar el bucle de validación cruzada

### Subtask:
Iterar a través de los pliegues definidos por la estrategia. En cada iteración: Dividir los datos en conjuntos de entrenamiento y validación según el pliegue actual. Crear una nueva instancia del modelo LSTM (es importante entrenar un modelo nuevo en cada pliegue para evitar la fuga de información). Entrenar el modelo en el conjunto de entrenamiento del pliegue actual. Evaluar el modelo entrenado en el conjunto de validación del pliegue actual, calculando métricas como MSE o RMSE. Almacenar las métricas de evaluación para cada pliegue.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np # numpy is already imported in previous cells

# Initialize a list to store evaluation results
cv_scores = []

# Loop through the splits defined by TimeSeriesSplit
for fold, (train_index, val_index) in enumerate(tscv.split(X_cv, y_cv)):
    print(f"--- Processing Fold {fold + 1}/{n_splits} ---")

    # Split data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_cv[train_index], X_cv[val_index]
    y_train_fold, y_val_fold = y_cv[train_index], y_cv[val_index]

    # --- Diagnostic Prints ---
    print(f"Shape of X_train_fold: {X_train_fold.shape}")
    print(f"Shape of y_train_fold: {y_train_fold.shape}")
    print(f"Shape of X_val_fold: {X_val_fold.shape}")
    print(f"Shape of y_val_fold: {y_val_fold.shape}")
    # -------------------------

    # Create a new instance of the LSTM model for each fold
    model = Sequential()
    # Using the shapes determined during the initial model definition
    model.add(LSTM(units=50, activation='relu', input_shape=(X_train_fold.shape[1], X_train_fold.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(units=y_train_fold.shape[1]))

    # Compile the new model instance
    model.compile(optimizer='adam', loss='mse')

    # Train the model on the training data for the current fold
    # verbose=0 to suppress detailed output for each epoch
    history = model.fit(X_train_fold, y_train_fold, epochs=50, batch_size=32, validation_data=(X_val_fold, y_val_fold), verbose=0)

    # Evaluate the trained model on the validation data of the current fold
    loss = model.evaluate(X_val_fold, y_val_fold, verbose=0)

    # Store the evaluation metric (e.g., MSE loss)
    cv_scores.append(loss)

    # Print the fold number and the loss for that fold
    print(f"Fold {fold + 1} Validation Loss (MSE): {loss}")

# Print the cross-validation scores
print("\nCross-validation MSE scores for each fold:", cv_scores)
print("Mean Cross-validation MSE:", np.mean(cv_scores))
print("Standard Deviation of Cross-validation MSE:", np.std(cv_scores))

## Analizar los resultados de la validación cruzada

### Subtask:
Calcular el promedio y la desviación estándar de las métricas de evaluación obtenidas en todos los pliegues para tener una estimación general del rendimiento del modelo y su variabilidad.


In [ ]:
# Calculate the mean of the cross-validation scores
mean_cv_score = np.mean(cv_scores)

# Calculate the standard deviation of the cross-validation scores
std_cv_score = np.std(cv_scores)

# Print the mean and standard deviation
print(f'Mean Cross-validation Score (MSE): {mean_cv_score}')
print(f'Standard Deviation of Cross-validation Score (MSE): {std_cv_score}')

## Summary:

### Data Analysis Key Findings

*   The data was successfully sorted chronologically and structured into sequences with a window size of 60 time steps, ready for time series cross-validation.
*   A `TimeSeriesSplit` object was configured with 5 splits to define the sequential train/test divisions.
*   The cross-validation loop successfully iterated through the 5 folds, training and evaluating a new LSTM model on each fold's training and validation data.
*   The Mean Squared Error (MSE) was calculated for each fold's validation set. The individual fold MSE scores were approximately 0.000069, 0.000076, 0.000115, 0.0681, and 0.0281.
*   The mean cross-validation MSE across the 5 folds is approximately 0.0281.
*   The standard deviation of the cross-validation MSE is approximately 0.0269, indicating some variability in the model's performance across the different time series splits.

### Insights or Next Steps

*   The relatively high standard deviation compared to the mean MSE suggests that the model's performance varies significantly depending on the specific time period used for validation. This could indicate the presence of non-stationarity or specific challenging periods in the time series data.
*   Further investigation into the folds with higher MSE might reveal patterns or characteristics in those data segments that the current model struggles with. Techniques like analyzing the residuals or visualizing predictions for those folds could be beneficial. Consider exploring more complex LSTM architectures, hyperparameter tuning, or incorporating external features that might explain the variability in performance.


## Análisis de Residuos

### Subtask:
Calcular los residuos (errores de predicción) del modelo en el conjunto de prueba y analizarlos.

In [ ]:
# Calculate the residuals
residuals = y_test - y_pred

print("Shape of residuals:", residuals.shape)

### Subtask:
Visualizar la distribución de los residuos para cada sonda.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get the column names for the probes
probe_names = df_scaled_numerical.columns.tolist()
num_probes = len(probe_names)

# Create subplots for the distribution of residuals for each probe
n_cols = 4
n_rows = (num_probes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten()

for i in range(num_probes):
    ax = axes[i]
    probe_name = probe_names[i]

    # Plot a histogram or distribution plot of the residuals for the current probe
    sns.histplot(residuals[:, i], ax=ax, kde=True)

    ax.set_title(f'Distribución de Residuos: {probe_name}')
    ax.set_xlabel('Residuo')
    ax.set_ylabel('Frecuencia')
    ax.grid(True)

# Hide any unused subplots
for j in range(num_probes, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle('Distribución de Residuos por Sonda (Conjunto de Prueba)', y=1.02)
plt.show()

## Realizar Pronósticos

### Subtask:
Utilizar el modelo entrenado para hacer pronósticos futuros.

**Reasoning**:
Prepare the last sequence of historical data as input and use the trained `final_model` to make a single-step forecast.

In [ ]:
# Asegúrate de tener el DataFrame df_scaled con los datos escalados y ordenados
# Si no lo tienes cargado, ejecuta la celda donde se creó (celda 418f686c)
# Asegúrate de que n_steps esté definido (de la celda aHPRKJMJS_Ym)
# Asegúrate de que el modelo entrenado (variable 'model') esté disponible (de la celda e404a3e5)

# Select only the numerical columns from the scaled DataFrame
df_scaled_numerical = df_scaled.drop(columns=['serie']).copy()

# Extrae la última secuencia de 'n_steps' puntos de datos
# El modelo fue entrenado con secuencias de n_steps
# n_steps should be defined from the sequence creation cell (aHPRKJMJS_Ym)
# Ensure n_steps is available in the environment. If not, define it here:
# n_steps = 60 # Example, use the actual value used for training

last_sequence = df_scaled_numerical.iloc[-n_steps:].values.astype(np.float32)

# The shape of the input for the model must be [samples, time steps, features]
# We have 1 sequence, n_steps time steps and num_features features (from df_scaled_numerical)
num_features = df_scaled_numerical.shape[1]
# Necesitamos remodelar la última secuencia para que tenga la forma [1, n_steps, num_features]
last_sequence_reshaped = last_sequence.reshape(1, n_steps, num_features)

# Hacer la predicción para el siguiente paso de tiempo
# Usamos el modelo entrenado (variable 'model')
predicted_scaled = model.predict(last_sequence_reshaped)

print("Shape of the predicted scaled value:", predicted_scaled.shape)
# La predicción es un array 2D con forma (1, num_features)
print("Predicted scaled value for the next time step:")
print(predicted_scaled)

## Invertir el escalado de las predicciones

### Subtask:
Convertir las predicciones escaladas de vuelta a la escala original.

In [ ]:
# Ensure the scaler object is available (it was created in cell 418f686c)
# If you restarted the notebook, you might need to re-run the scaling cell.

# The inverse_transform method expects a 2D array, so we need to reshape if it's 1D
# Our predicted_scaled is already 2D with shape (1, num_features), which is correct.

# Inverse transform the scaled predictions
predicted_original_scale = scaler.inverse_transform(predicted_scaled)

print("Predicted values in original scale for the next time step:")
# predicted_original_scale is a 2D array with shape (1, num_features), so print the first row
print(predicted_original_scale[0])

In [ ]:
# Asegúrate de que el DataFrame original 'df' esté cargado (con los datos del rango de años deseado, ej. 2019-2024).
# Si no es así, ejecuta la celda donde lo cargaste inicialmente (celda n3nrda8PCljf o similar, ajustando la ruta si es necesario).

# Calcula el rango (mínimo y máximo) de la columna 'intensidad_campo' en el DataFrame original
if 'df' in locals():
    min_intensity = df['intensidad_campo'].min()
    max_intensity = df['intensidad_campo'].max()
    print(f"Rango de Intensidad de Campo en los datos originales (2019-2024): [{min_intensity}, {max_intensity}]")

    # Obtén los valores pronosticados en escala original (de la celda 716b59a2)
    # Asegúrate de que predicted_original_scale esté disponible
    if 'predicted_original_scale' in locals():
        print("\nValores pronosticados en escala original para el siguiente paso de tiempo:")
        print(predicted_original_scale[0])

        # Interpretación básica: comparar con el rango
        print("\nInterpretación:")
        print(f"El rango de intensidad de campo en los datos históricos originales (2019-2024) va de {min_intensity:.4f} a {max_intensity:.4f}.")
        print(f"Los valores pronosticados para el siguiente paso de tiempo (para las {len(predicted_original_scale[0])} sondas restantes) son:")
        # Imprimir cada predicción individualmente para facilitar la comparación
        # Asegúrate de que probe_names esté disponible (de la celda 4922023f o similar, que usa df_scaled_numerical)
        if 'probe_names' in locals() and len(probe_names) == len(predicted_original_scale[0]):
            for i, pred_value in enumerate(predicted_original_scale[0]):
                 print(f"- {probe_names[i]}: {pred_value:.4f}")
        else:
             print("No se pudieron obtener los nombres de las sondas o no coinciden con las predicciones.")
             print(predicted_original_scale[0])


        # Puedes comentar si las predicciones caen dentro o fuera del rango histórico
        all_within_range = all(min_intensity <= pred <= max_intensity for pred in predicted_original_scale[0])

        if all_within_range:
            print("\nTodos los valores pronosticados caen dentro del rango observado en los datos históricos (2019-2024).")
        else:
            print("\nAl menos uno de los valores pronosticados cae fuera del rango observado en los datos históricos (2019-2024).")

    else:
        print("\npredicted_original_scale no está disponible. Por favor, ejecuta las celdas de pronóstico de un solo paso (cbfac208 y 716b59a2).")

else:
    print("El DataFrame original 'df' no está cargado. Por favor, cárgalo (celda n3nrda8PCljf) para obtener el rango.")

# Pronósticos multi-paso
Genera código Python para realizar pronósticos multi-paso utilizando un modelo LSTM entrenado, siguiendo los pasos del plan proporcionado.

## Preparar la secuencia de entrada inicial

### Subtask:
Obtener la última secuencia de datos históricos (`n_steps`) del conjunto de datos escalado, similar a cómo lo hicimos para el pronóstico de un solo paso.


**Reasoning**:
Extract the last sequence of historical data from the scaled numerical DataFrame and reshape it for multi-step forecasting input.



**Reasoning**:
Implement a loop to generate multi-step forecasts by repeatedly predicting the next step and using it as part of the input sequence for the subsequent prediction.



**Reasoning**:
Inverse transform the multi-step scaled forecasts back to the original data scale using the fitted scaler.



In [ ]:
# Calcula el rango (mínimo y máximo) de la columna 'intensidad_campo' en el DataFrame original
if 'df' in locals():
    min_intensity = df['intensidad_campo'].min()
    max_intensity = df['intensidad_campo'].max()
    print(f"Rango de Intensidad de Campo en los datos originales: [{min_intensity}, {max_intensity}]")

    # Obtén los valores pronosticados en escala original (de la celda 716b59a2)
    # Asegúrate de que predicted_original_scale esté disponible
    if 'predicted_original_scale' in locals():
        print("\nValores pronosticados en escala original para el siguiente paso de tiempo:")
        print(predicted_original_scale[0])

        # Interpretación básica: comparar con el rango
        print("\nInterpretación:")
        print(f"El rango de intensidad de campo en los datos históricos originales va de {min_intensity:.4f} a {max_intensity:.4f}.")
        print("Los valores pronosticados para el siguiente paso de tiempo son:")
        # Imprimir cada predicción individualmente para facilitar la comparación
        probe_names = df_scaled_numerical.columns.tolist() # Assuming df_scaled_numerical is available from earlier steps
        for i, pred_value in enumerate(predicted_original_scale[0]):
             print(f"- {probe_names[i]}: {pred_value:.4f}")

        # Puedes comentar si las predicciones caen dentro o fuera del rango histórico
        all_within_range = all(min_intensity <= pred <= max_intensity for pred in predicted_original_scale[0])

        if all_within_range:
            print("\nTodos los valores pronosticados caen dentro del rango observado en los datos históricos.")
        else:
            print("\nAl menos uno de los valores pronosticados cae fuera del rango observado en los datos históricos.")

    else:
        print("\npredicted_original_scale no está disponible. Por favor, ejecuta la celda 716b59a2 para obtener los valores pronosticados en escala original.")

else:
    print("El DataFrame original 'df' no está cargado. Por favor, cárgalo para obtener el rango.")

## Preparar la secuencia de entrada inicial

### Subtask:
Obtener la última secuencia de datos históricos (`n_steps`) del conjunto de datos escalado, similar a cómo lo hicimos para el pronóstico de un solo paso.

**Reasoning**:
Extract the last sequence of historical data from the scaled numerical DataFrame and reshape it for multi-step forecasting input.

In [ ]:
# Select only the numerical columns from the scaled DataFrame
df_scaled_numerical = df_scaled.drop(columns=['serie']).copy()

# Extract the last sequence of 'n_steps' data points
# The model was trained with sequences of n_steps
n_steps = 60 # Ensure n_steps is defined, using the same value as before
last_sequence = df_scaled_numerical.iloc[-n_steps:].values.astype(np.float32)

# The shape of the input for the model must be [samples, time steps, features]
# For multi-step forecasting, we will feed the predicted value back as input.
# But for the very first step of multi-step forecasting, the input is the last known sequence.
# We need to reshape the last sequence to have the shape [1, n_steps, num_features]
last_sequence_reshaped = last_sequence.reshape(1, n_steps, last_sequence.shape[1])

print("Shape of the last historical sequence for multi-step forecasting:", last_sequence_reshaped.shape)

**Reasoning**:
Implement a loop to generate multi-step forecasts by repeatedly predicting the next step and using it as part of the input sequence for the subsequent prediction.

In [ ]:
# Define the number of future steps to forecast
n_forecast_steps = 180 # Changed to approximately 6 months

# Initialize a list to store the multi-step forecasts
multi_step_forecasts = []

# Use the last historical sequence as the initial input for forecasting
current_input_sequence = last_sequence_reshaped

# Loop to generate forecasts for the specified number of steps
for i in range(n_forecast_steps):
    # Make a prediction for the next time step using the current input sequence
    predicted_scaled_step = model.predict(current_input_sequence)

    # Store the predicted step (in scaled format)
    multi_step_forecasts.append(predicted_scaled_step[0]) # Append the prediction for this step

    # Prepare the input sequence for the next prediction
    # Remove the oldest time step from the current input sequence
    # Add the newly predicted step to the end of the sequence
    current_input_sequence = np.append(current_input_sequence[:, 1:, :], predicted_scaled_step.reshape(1, 1, predicted_scaled_step.shape[1]), axis=1)

    # Optional: Print progress
    if (i + 1) % 10 == 0:
        print(f"Generated forecast for step {i + 1}/{n_forecast_steps}. Shape of next input sequence: {current_input_sequence.shape}")


# Convert the list of forecasts into a NumPy array
multi_step_forecasts_scaled = np.array(multi_step_forecasts)

print("\nShape of multi-step forecasts (scaled):", multi_step_forecasts_scaled.shape)
# The shape should be (n_forecast_steps, num_features)

**Reasoning**:
Inverse transform the multi-step scaled forecasts back to the original data scale using the fitted scaler.

In [ ]:
# Ensure the scaler object is available (it was created in cell 418f686c)
# If you restarted the notebook, you might need to re-run the scaling cell.

# Inverse transform the multi-step scaled forecasts
multi_step_forecasts_original_scale = scaler.inverse_transform(multi_step_forecasts_scaled)

print("Multi-step forecasts in original scale:")
print(multi_step_forecasts_original_scale)

## Visualización de Pronósticos Multi-Paso

### Subtask:
Visualizar los pronósticos multi-paso junto con los datos históricos recientes.

**Reasoning**:
Plot the last part of the historical data and the multi-step forecasts together to visualize the predictions in context.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # Ensure numpy is imported for nan check

# Ensure df_scaled_numerical is available (from cell 2719cdac or earlier)
# Ensure multi_step_forecasts_original_scale is available (from cell 571237b3)
# Ensure n_steps is defined (from cell aHPRKJMJS_Ym or earlier)
# Ensure probe_names is available (from cell 4922023f or earlier)
# Ensure df_final is available (from cell n3nrda8PCljf or the cell that creates it, before final imputation)


# --- Identify probes with data up to 2023 (reusing logic) ---
if 'df_final' not in locals():
    print("df_final not found. Cannot identify probes with data up to 2023.")
    # If df_final is not available, default to plotting all probes (or handle error)
    probes_to_plot = None
else:
    # Make a temporary copy of df_final and convert "N/A" to NaN to find last non-null dates
    df_temp_dates = df_final.replace("N/A", np.nan).copy()

    # Ensure 'serie' is datetime
    df_temp_dates['serie'] = pd.to_datetime(df_temp_dates['serie'])
    df_temp_dates = df_temp_dates.set_index('serie')

    # Find the last non-null index (date) for each column
    last_valid_dates = df_temp_dates.apply(lambda x: x.last_valid_index())

    # Filter probes whose last valid date is within the year 2023
    probes_data_until_2023_series = last_valid_dates[
        (last_valid_dates.dt.year == 2023)
    ]

    # --- User Input: Set probes_to_plot based on identified probes ---
    # Set probes_to_plot to the list of probes identified as having data until 2023
    probes_to_plot = probes_data_until_2023_series.index.tolist()

    if not probes_to_plot:
        print("No probes found with data up to 2023. Plotting all probes instead.")
        probes_to_plot = None # Default to plotting all if none found
    else:
        print(f"Identified probes with data up to 2023: {probes_to_plot}")

# --------------------------------------------------------------------------


# Get the last part of the historical data in original scale for plotting context
# Increase the historical plot period for better visibility
plot_history_steps = n_steps * 3 # Increased from 2*n_steps

# Ensure df_final_imputed is available and in original scale (from cell 3c8b12c3)
if 'df_final_imputed' not in locals():
    print("df_final_imputed not found. Please run the data loading and imputation cells.")
else:
    # Select the numerical columns from the imputed original scale data
    df_original_numerical = df_final_imputed.drop(columns=['serie']).copy()

    # Get the last historical data points to plot
    historical_data_to_plot = df_original_numerical.iloc[-plot_history_steps:]

    # Create a time index for the historical data
    historical_dates = pd.to_datetime(df_final_imputed['serie'].iloc[-plot_history_steps:])

    # Create a time index for the multi-step forecasts
    # Assuming daily frequency, the forecast starts the day after the last historical date
    last_historical_date = pd.to_datetime(df_final_imputed['serie'].iloc[-1])
    forecast_dates = pd.date_range(start=last_historical_date + pd.Timedelta(days=1),
                                   periods=len(multi_step_forecasts_original_scale),
                                   freq='D')

    # --- Diagnostic Check for NaNs ---
    print("Checking for NaNs in historical data to plot:", historical_data_to_plot.isnull().sum().sum())
    print("Checking for NaNs in multi_step_forecasts_original_scale:", np.isnan(multi_step_forecasts_original_scale).sum())
    # ---------------------------------

    # Determine which probes to plot and filter the data accordingly
    if probes_to_plot is not None and len(probes_to_plot) > 0:
        try:
            # Filter historical data and forecasts based on the specified probes
            # Ensure the probes_to_plot are actually in the columns of historical_data_to_plot
            valid_probes_to_plot = [p for p in probes_to_plot if p in historical_data_to_plot.columns]
            if len(valid_probes_to_plot) != len(probes_to_plot):
                 print("Warning: Some specified probes not found in historical data columns.")
                 probes_to_plot = valid_probes_to_plot # Update probes_to_plot to valid ones

            historical_data_filtered = historical_data_to_plot[probes_to_plot]
            # Get the indices of the selected probes from the original probe_names list
            # Need to get indices from the full list of probe_names derived from df_scaled_numerical
            if 'probe_names' in locals():
                 probe_indices = [probe_names.index(probe) for probe in probes_to_plot]
                 forecasts_filtered = multi_step_forecasts_original_scale[:, probe_indices]
                 probe_names_filtered = probes_to_plot
                 print(f"Plotting selected probes: {probe_names_filtered}")
            else:
                 print("Error: probe_names not found. Cannot filter forecasts correctly.")
                 # Default to plotting all probes if there's an error
                 historical_data_filtered = historical_data_to_plot
                 forecasts_filtered = multi_step_forecasts_original_scale
                 probe_names_filtered = df_original_numerical.columns.tolist()
                 print("Plotting all probes instead due to error.")


        except ValueError as e:
            print(f"Error during filtering based on probes_to_plot. Error: {e}")
            # Default to plotting all probes if there's an error
            historical_data_filtered = historical_data_to_plot
            forecasts_filtered = multi_step_forecasts_original_scale
            probe_names_filtered = df_original_numerical.columns.tolist()
            print("Plotting all probes instead due to error.")
    else:
        # If probes_to_plot is None or empty, plot all probes
        historical_data_filtered = historical_data_to_plot
        forecasts_filtered = multi_step_forecasts_original_scale
        probe_names_filtered = df_original_numerical.columns.tolist()
        print("Plotting all probes.")


    # Determine the number of probes to plot
    num_probes_to_plot = len(probe_names_filtered)

    # Create subplots for each probe
    n_cols = min(num_probes_to_plot, 4) # Use max 4 columns
    n_rows = (num_probes_to_plot + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5)) # Adjusted figsize
    # Handle case where there's only one subplot
    if num_probes_to_plot == 1:
        axes = [axes]
    else:
        axes = axes.flatten()


    for i in range(num_probes_to_plot):
        ax = axes[i]
        probe_name = probe_names_filtered[i]

        # Get the data for the current probe
        # Access data from filtered dataframes/arrays
        historical_data_probe = historical_data_filtered.iloc[:, i]
        forecasts_probe = forecasts_filtered[:, i]


        # Plot historical data
        ax.plot(historical_dates, historical_data_probe, label='Datos Históricos')

        # Plot multi-step forecasts
        ax.plot(forecast_dates, forecasts_probe, label='Pronósticos', linestyle='--')

        ax.set_title(f'Pronóstico Multi-Paso: {probe_name}')
        ax.set_xlabel('Fecha')
        ax.set_ylabel('Intensidad de Campo (Original Scale)')
        ax.legend()
        ax.grid(True)
        plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for readability

    # Hide any unused subplots
    for j in range(num_probes_to_plot, len(axes)):
        fig.delaxes(axes[j])


    plt.tight_layout()
    plt.suptitle('Pronósticos Multi-Paso vs. Datos Históricos Recientes por Sonda', y=1.02)
    plt.show()

## Identificar la última fecha con datos medidos por sonda

### Subtask:
Encontrar la última fecha no nula para cada columna (sonda) en el DataFrame `df_final` (antes de la imputación completa).

**Reasoning**:
Identify the last non-null index (date) for each column in the df_final DataFrame to understand the extent of measured data for each probe.

# Dashboard

In [ ]:
!pip install scikit-optimize
!pip install streamlit -q #instalación de librerías
!pip install pyngrok
!pip install optuna
!pip install streamlit pandas matplotlib seaborn scikit-learn pyngrok kagglehub
!pip install pyngrok streamlit --quiet

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from io import StringIO
import os
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, mean_absolute_error

# --- Page Config ---
st.set_page_config(page_title="Análisis de Intensidad de Campo", layout="wide")
st.title("Dashboard de Análisis de Intensidad de Campo Electromagnético")

# --- Constants and Paths ---
MODEL_PATH = 'lstm_model.h5'
SCALER_PATH = 'scaler.pkl'
PROBE_NAMES_PATH = 'probe_names.pkl'
HISTORY_PATH = 'history.pkl'
DATA_PATH = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/P_CEM/Bog_promedios_diarios_2019_2024_filtered_2023_probes.csv'

# --- Utility Functions ---

@st.cache_data
def load_data(data_path):
    st.info("Cargando datos de promedios diarios...")
    try:
        df = pd.read_csv(data_path)
        df['serie'] = pd.to_datetime(df['serie'])
    except FileNotFoundError:
        st.error(f"Error: No se encontró el archivo en {data_path}")
        return None
    st.success("Datos cargados exitosamente.")
    return df

@st.cache_data
def run_preprocessing(_df):
    """
    Ejecuta todos los pasos de preprocesamiento y devuelve los dataframes y objetos necesarios.
    Se cachea para que solo se ejecute una vez.
    """
    st.info("Ejecutando preprocesamiento de datos...")
    df = _df.copy()
    start_date = '2019-01-01'
    end_date = '2023-11-01'
    mask = (df['serie'] >= start_date) & (df['serie'] <= end_date)
    df_filtered = df.loc[mask].copy()
    df_filtered.replace("N/A", np.nan, inplace=True)
    probe_columns = [col for col in df_filtered.columns if 'Bogota' in col]
    for col in probe_columns:
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

    missing_percentage = df_filtered.isnull().sum() / len(df_filtered) * 100
    cols_to_drop = missing_percentage[missing_percentage > 60].index.tolist()
    df_probes_dropped = df_filtered.drop(columns=cols_to_drop)

    df_temp_dates = df_probes_dropped.set_index('serie')
    last_valid_dates = df_temp_dates.apply(lambda x: x.last_valid_index())
    probes_until_2023 = last_valid_dates[last_valid_dates.dt.year == 2023].index.tolist()
    final_probes = ['serie'] + probes_until_2023
    df_final_selection = df_probes_dropped[final_probes]

    df_imputed = df_final_selection.copy()
    for col in probes_until_2023:
        median_val = df_imputed[col].median()
        df_imputed[col].fillna(median_val, inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    df_to_scale = df_imputed[probes_until_2023]
    scaled_data = scaler.fit_transform(df_to_scale)
    df_scaled = pd.DataFrame(scaled_data, columns=probes_until_2023)
    df_scaled['serie'] = pd.to_datetime(df_imputed['serie'].values)

    st.success("Preprocesamiento completado.")
    return df_imputed, df_scaled, scaler, probes_until_2023

def create_sequences(data, n_steps):
    X, y = [], []
    numerical_data = data.values
    for i in range(len(data) - n_steps):
        seq_x = numerical_data[i:(i + n_steps), :]
        seq_y = numerical_data[i + n_steps, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

@st.cache_resource
def get_or_train_model(X_train, y_train, X_val, y_val, _scaler, probe_names):
    if all(os.path.exists(p) for p in [MODEL_PATH, SCALER_PATH, PROBE_NAMES_PATH, HISTORY_PATH]):
        st.info("Cargando modelo y recursos pre-entrenados...")
        try:
            model = load_model(MODEL_PATH, custom_objects={'mse': MeanSquaredError()})
            with open(SCALER_PATH, 'rb') as f:
                scaler = pickle.load(f)
            with open(PROBE_NAMES_PATH, 'rb') as f:
                saved_probe_names = pickle.load(f)
            with open(HISTORY_PATH, 'rb') as f:
                history = pickle.load(f)
            st.success("Modelo y recursos cargados desde archivos.")
            return model, scaler, saved_probe_names, history
        except Exception as e:
            st.warning(f"No se pudieron cargar los archivos guardados ({e}). Re-entrenando el modelo...")

    st.info("Entrenando un nuevo modelo LSTM...")
    model = Sequential([
        LSTM(units=75, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.3),
        LSTM(units=75, activation='relu'),
        Dropout(0.3),
        Dense(units=y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')

    with st.spinner("El modelo se está entrenando. Esto puede tardar unos minutos..."):
        history_obj = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), verbose=0)
        history_dict = history_obj.history

    model.save(MODEL_PATH)
    with open(SCALER_PATH, 'wb') as f:
        pickle.dump(_scaler, f)
    with open(PROBE_NAMES_PATH, 'wb') as f:
        pickle.dump(probe_names, f)
    with open(HISTORY_PATH, 'wb') as f:
        pickle.dump(history_dict, f)

    st.success("Modelo entrenado y guardado exitosamente.")
    return model, _scaler, probe_names, history_dict


# --- Visualization Functions ---
def plot_time_series(df, probe_name):
    fig, ax = plt.subplots(figsize=(12, 6))
    data_to_plot = df[['serie', probe_name]].dropna()
    ax.plot(data_to_plot['serie'], data_to_plot[probe_name])
    ax.set_title(f'Serie de Tiempo: {probe_name}')
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Intensidad de Campo')
    ax.grid(True)
    st.pyplot(fig)

def plot_boxplot(df, probe_name):
    fig, ax = plt.subplots(figsize=(8, 6))
    data_to_plot = df[probe_name].dropna()
    if not data_to_plot.empty:
        sns.boxplot(y=data_to_plot, ax=ax)
        ax.set_title(f'Boxplot de Intensidad de Campo: {probe_name}')
        ax.set_ylabel('Intensidad de Campo')
        st.pyplot(fig)

def plot_training_history(history):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(history['loss'], label='Pérdida de Entrenamiento')
    ax.plot(history['val_loss'], label='Pérdida de Validación')
    ax.set_title('Historial de Pérdida del Modelo (Curvas de Aprendizaje)')
    ax.set_xlabel('Época')
    ax.set_ylabel('Pérdida (MSE)')
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

def plot_predictions_vs_actuals(y_true, y_pred, probe_name, probe_names):
    try:
        probe_index = probe_names.index(probe_name)
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(y_true[:, probe_index], label='Valores Reales')
        ax.plot(y_pred[:, probe_index], label='Predicciones del Modelo', linestyle='--')
        ax.set_title(f'Predicciones vs. Valores Reales para: {probe_name}')
        ax.set_xlabel('Paso de Tiempo (en conjunto de prueba)')
        ax.set_ylabel('Intensidad de Campo (Escala Original)')
        ax.legend()
        ax.grid(True)
        st.pyplot(fig)
    except (ValueError, IndexError):
        st.error(f"No se pudo encontrar la sonda '{probe_name}' en la lista de sondas del modelo.")

def make_multistep_forecast(model, scaler, df_scaled, probe_names, n_forecast_steps, n_steps):
    st.info(f"Generando pronóstico para los próximos {n_forecast_steps} días...")
    numerical_data = df_scaled[probe_names].values
    last_sequence = numerical_data[-n_steps:]
    input_sequence = last_sequence.reshape(1, n_steps, len(probe_names))

    forecasts_scaled = []
    progress_bar = st.progress(0)
    for i in range(n_forecast_steps):
        pred_scaled = model.predict(input_sequence, verbose=0)
        forecasts_scaled.append(pred_scaled[0])
        input_sequence = np.append(input_sequence[:, 1:, :], [[pred_scaled[0]]], axis=1)
        progress_bar.progress((i + 1) / n_forecast_steps)

    forecasts_original = scaler.inverse_transform(np.array(forecasts_scaled))
    st.success("Pronóstico generado.")
    return forecasts_original

def plot_all_forecasts(historical_df, forecasts_original, probe_names, n_steps):
    st.subheader("Visualización de Pronósticos por Sonda")
    st.write("Cada gráfico muestra los datos históricos recientes y el pronóstico a futuro para una sonda individual.")

    num_probes = len(probe_names)
    n_cols = 3
    n_rows = (num_probes + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5), squeeze=False)
    axes = axes.flatten()

    for i, probe_name in enumerate(probe_names):
        ax = axes[i]
        history_to_plot = historical_df.iloc[-n_steps*2:]
        ax.plot(history_to_plot['serie'], history_to_plot[probe_name], label='Datos Históricos')

        last_historical_date = historical_df['serie'].iloc[-1]
        forecast_dates = pd.date_range(start=last_historical_date + pd.Timedelta(days=1), periods=len(forecasts_original))

        forecast_values = forecasts_original[:, i]
        ax.plot(forecast_dates, forecast_values, label='Pronóstico', linestyle='--', color='red')

        ax.set_title(f'Pronóstico para: {probe_name}')
        ax.set_xlabel('Fecha')
        ax.set_ylabel('Intensidad')
        ax.legend()
        ax.grid(True)
        for label in ax.get_xticklabels():
            label.set_rotation(45)
            label.set_ha('right')

    for j in range(num_probes, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    st.pyplot(fig)


# --- Main App Flow ---
if __name__ == '__main__':
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')

    df_raw = load_data(DATA_PATH)

    if df_raw is not None:
        # --- Preprocessing Step (runs once and is cached) ---
        df_imputed, df_scaled, scaler, final_probe_names = run_preprocessing(df_raw)

        # Store necessary items in session state
        st.session_state['df_imputed'] = df_imputed
        st.session_state['df_scaled'] = df_scaled
        st.session_state['scaler'] = scaler
        st.session_state['final_probe_names'] = final_probe_names

        # --- UI Tabs ---
        tab1, tab2, tab3, tab4 = st.tabs([
            "1. Exploración", "2. Preprocesamiento", "3. Entrenamiento y Evaluación", "4. Pronóstico"
        ])

        with tab1:
            st.header("1. Exploración de Datos Inicial")
            st.write("Visualizaciones de los datos crudos para entender la distribución y comportamiento de cada sonda.")
            st.dataframe(df_raw.head())
            all_probe_names = [col for col in df_raw.columns if 'Bogota' in col]
            probe_to_explore = st.selectbox("Selecciona una Sonda para Explorar", options=all_probe_names, key="explore_select")
            if probe_to_explore:
                col1, col2 = st.columns(2)
                with col1:
                    plot_time_series(df_raw, probe_to_explore)
                with col2:
                    plot_boxplot(df_raw, probe_to_explore)

        with tab2:
            st.header("2. Preprocesamiento Detallado")
            st.write("Pasos aplicados para limpiar y preparar los datos para el modelo.")
            st.markdown("##### Datos Después de la Imputación (Escala Original)")
            st.dataframe(st.session_state['df_imputed'].head())
            st.markdown("##### Datos Después de la Normalización (Listos para el Modelo)")
            st.dataframe(st.session_state['df_scaled'].head())

        with tab3:
            st.header("3. Entrenamiento y Evaluación del Modelo")
            n_steps = st.slider("Pasos de tiempo (días):", 30, 90, 60, key="n_steps_slider_train")

            X, y = create_sequences(st.session_state.df_scaled[final_probe_names], n_steps)
            X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            model, _, _, history = get_or_train_model(X_train, y_train, X_val, y_val, st.session_state.scaler, final_probe_names)
            st.session_state['model'] = model

            if model:
                st.subheader("Evaluación del Modelo")
                if st.button("▶️ Evaluar Modelo"):
                    with st.spinner("Evaluando..."):
                        y_pred_scaled = model.predict(X_test)
                        y_pred_original = scaler.inverse_transform(y_pred_scaled)
                        y_test_original = scaler.inverse_transform(y_test)
                        rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
                        mae = mean_absolute_error(y_test_original, y_pred_original)
                        st.session_state.update({'rmse': rmse, 'mae': mae, 'y_pred_original': y_pred_original, 'y_test_original': y_test_original})
                    st.success("Evaluación completada.")

                if 'rmse' in st.session_state:
                    col1, col2 = st.columns(2)
                    col1.metric("RMSE (Error General)", f"{st.session_state['rmse']:.4f} V/m")
                    col2.metric("MAE (Error Promedio)", f"{st.session_state['mae']:.4f} V/m")

                st.subheader("Curvas de Aprendizaje")
                if history:
                    plot_training_history(history)

                st.subheader("Predicciones vs. Valores Reales")
                probe_to_eval = st.selectbox("Selecciona Sonda para Comparar Predicciones", options=final_probe_names, key="eval_select")
                if 'y_pred_original' in st.session_state:
                    plot_predictions_vs_actuals(st.session_state['y_test_original'], st.session_state['y_pred_original'], probe_to_eval, final_probe_names)

        with tab4:
            st.header("4. Pronóstico a Futuro")
            if 'model' in st.session_state:
                n_forecast_steps = st.sidebar.slider("Días a pronosticar:", 30, 180, 90, key="forecast_days_slider")
                if st.sidebar.button("🚀 Generar Pronóstico"):
                    with st.spinner("Generando pronósticos..."):
                        forecast_results = make_multistep_forecast(st.session_state.model, st.session_state.scaler, st.session_state.df_scaled, st.session_state.final_probe_names, n_forecast_steps, st.session_state.get('n_steps', 60))
                        st.session_state['forecast_results'] = forecast_results
                    st.success("Pronóstico generado.")

                if 'forecast_results' in st.session_state:
                    plot_all_forecasts(st.session_state.df_imputed, st.session_state.forecast_results, st.session_state.final_probe_names, st.session_state.get('n_steps', 60))
            else:
                st.warning("El modelo debe ser entrenado primero en la Pestaña 3.")

In [ ]:
!wget -q -O - https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 > cloudflared-linux-amd64 && chmod +x cloudflared-linux-amd64
!streamlit run app.py &>/content/logs.txt &
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

import time
import re

time.sleep(10) # Increased wait time for tunnel to establish

# Read the cloudflared log to find the public URL
try:
    with open('nohup.out', 'r') as f:
        for line in f:
            # Search for the trycloudflare.com URL in the log
            match = re.search(r'https?://\S+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                print(f"✅ ¡Tu dashboard está listo! Ábrelo aquí: {url}")
                break
except FileNotFoundError:
    print("No se encontró el archivo de log 'nohup.out'. Revisa los logs para más detalles.")


In [ ]:
!kill -9 <PID>